In [1]:
%matplotlib inline
import math, copy
import matplotlib.pyplot as plt
import numpy as np
# import pandas as pd
# import SimpleITK as sitk
# import tomopy

from IPython.display import display
from matplotlib import cm
# from scipy import ndimage
# from skimage.metrics import structural_similarity as ssim
# from skimage.util import compare_images
# from sklearn.metrics import mean_absolute_error, mean_squared_error

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

import tigre
import numpy as np
from tigre.utilities import sample_loader
from tigre.utilities import CTnoise
import tigre.algorithms as algs


import itk


In [2]:
# nb_lines=128


# pixel_spacing_in_micrometre = 1.9;
# pixel_spacing_in_mm = pixel_spacing_in_micrometre * 1e-3;
# number_of_projections = 900;
# angular_span_in_degrees = 180.0;
# angular_step = angular_span_in_degrees / number_of_projections;
# theta_deg = np.linspace(0.,
#                     angular_span_in_degrees,
#                     number_of_projections,
#                     endpoint=False);
# theta_rad = theta_deg / 180.0 * math.pi;




# print(theta_rad.shape, "angles from", theta_rad[0], "to", theta_rad[-1])


# # Lets create a geomtry object
# geo = tigre.geometry()

# # VARIABLE                                   DESCRIPTION                    UNITS
# # -------------------------------------------------------------------------------------
# # Distances
# geo.DSD = 140 * 1000 + 80  # Distance Source Detector      (mm)
# geo.DSO = 140 * 1000  # Distance Source Origin        (mm)
# # Detector parameters
# geo.nDetector = np.array([nb_lines, 1024])  # number of pixels              (px)
# geo.dDetector = np.array([0.0019, 0.0019])  # size of each pixel            (mm)
# geo.sDetector = geo.nDetector * geo.dDetector  # total size of the detector    (mm)

# # Image parameters
# geo.nVoxel = np.array([nb_lines, 1024, 1024])  # number of voxels              (vx)
# geo.sVoxel = np.array([1024 * 0.0019, 1024 * 0.0019, 0.0019])  # total size of the image       (mm)
# geo.dVoxel = geo.sVoxel / geo.nVoxel  # size of each voxel            (mm)
# # Offsets
# geo.offOrigin = np.array([0, 0, 0])  # Offset of image from origin   (mm)
# geo.offDetector = np.array([0, 0])  # Offset of Detector            (mm)
# # These two can be also defined
# # per angle

# # Auxiliary
# geo.accuracy = 0.5  # Variable to define accuracy of
# # 'interpolated' projection
# # It defines the amoutn of
# # samples per voxel.
# # Recommended <=0.5             (vx/sample)

# # Optional Parameters
# # There is no need to define these unless you actually need them in your
# # reconstruction


# geo.COR = 0  # y direction displacement for
# # centre of rotation
# # correction                   (mm)
# # This can also be defined per
# # angle

# geo.rotDetector = np.array([0, 0, 0])  # Rotation of the detector, by
# # X,Y and Z axis respectively. (rad)
# # This can also be defined per
# # angle

# geo.mode = "parallel"  # Or 'parallel'. Geometry type.

# #%% Print the Geometry
# print(geo)
# #%% Alternatively,
# # # if you are just experimenting with TIGRE and you dont care too much about the geometry, you can generate default geometries as:
# # geo = tigre.geometry_default()  # Default cone beam geometry
# # geo = tigre.geometry(
# #     mode="cone", default=True
# # )  # default=True calculates all parameters for the geometry for you, so you can do:
# # geo = tigre.geometry(
# #     mode="cone", default=True, nVoxel=np.array([256, 256, 256])
# # )  # This will calculate a reasonable geometry for this number of voxels
# # geo = tigre.geometry(
# #     mode="cone", default=True, high_resolution=True
# # )  # high_resolution will result on an image 512^3, while false will result on a 128^3 image. nVoxel overrides these values.
# # geo = tigre.geometry(
# #     mode="parallel", nVoxel=np.array([1, 1024, 1024])
# # )  # Parallel beam geometry does not require anything other than the image size.

# #%% Plot your geometry
# # geo = tigre.geometry_default()  # Default cone beam geometry
# tigre.plot_geometry(geo, angle=-np.pi / 6)
# # geo = tigre.geometry_default(high_resolution=False)

In [7]:
CPUImageType = itk.Image[itk.F,2]

number_of_projections = 900
reference_normalised_projections = np.fromfile("sino.raw", dtype=np.float32);
reference_normalised_projections.shape = [
    number_of_projections,
    1, 
    int(reference_normalised_projections.shape[0] / number_of_projections)
];

projections = -np.log(reference_normalised_projections);


# ReaderType = itk.ImageFileReader[CPUImageType]
# reader = ReaderType.New();
# reader.SetFileName("outputs/reference_sinogram.mha");
# reader.Update();



# projection_data = itk.GetArrayFromImage(reader.GetOutput())
# projection_data.shape = [
#     projection_data.shape[1],
#     projection_data.shape[0],
#     projection_data.shape[2]
# ]

print(projections.shape)
geo = tigre.geometry_default()

geo.mode = "cone"  # Or 'parallel'. Geometry type.
geo.DSD = 140 * 1000 + 80  # Distance Source Detector      (mm)
geo.DSO = 140 * 1000  # Distance Source Origin        (mm)


# print("Writing output image...")
# WriterType = itk.ImageFileWriter[CPUImageType]
# writer = WriterType.New();
# writer.SetFileName("projection_data.mha");
# writer.SetInput(itk.GetImageFromArray(projection_data));
# writer.Update();

print("geo.nDetector", geo.nDetector)
print("geo.dDetector", geo.dDetector)
print("geo.sDetector", geo.sDetector)

print("geo.nVoxel", geo.nVoxel)
print("geo.sVoxel", geo.sVoxel)
print("geo.dVoxel", geo.dVoxel)

geo.nDetector = np.array([1, 1024])
geo.dDetector = np.array([.0019, .0019])
geo.sDetector = geo.nDetector * geo.dDetector  # total size of the detector    (mm)

geo.nVoxel = np.array([1, 1024, 1024])
geo.dVoxel = np.array([.0019, .0019, .0019])
geo.sVoxel = geo.nVoxel * geo.dVoxel  # total size of the detector    (mm)


(900, 1, 1024)
geo.nDetector [512 512]
geo.dDetector [0.8 0.8]
geo.sDetector [409.6 409.6]
geo.nVoxel [512 512 512]
geo.sVoxel [256 256 256]
geo.dVoxel [0.5 0.5 0.5]


In [4]:


# #%% Reconstruct image using OS-SART and FDK

# FDK
imgFDK = algs.fdk(projections, geo, theta_rad)
imgFDK = copy.deepcopy(imgFDK.T)

print("Writing output image...")
WriterType = itk.ImageFileWriter[CPUImageType]
writer = WriterType.New();
writer.SetFileName("imgFDK.mha");
writer.SetInput(itk.GetImageFromArray(imgFDK));
writer.Update();


# OS-SART

# niter = 50
# imgOSSART = algs.ossart(projections, geo, theta_rad, niter)

# print("Writing output image...")
# WriterType = itk.ImageFileWriter[CPUImageType]
# writer = WriterType.New();
# writer.SetFileName("imgOSSART.mha");
# writer.SetInput(itk.GetImageFromArray(imgOSSART.T));
# writer.Update();

#%% Show the results
print(imgFDK.shape)
plt.imshow(imgFDK, cmap="gray")
# plt.imshow(imgOSSART[int(imgFDK.shape[0] / 2)].T, cmap="gray")


NameError: name 'geo' is not defined

In [ ]:
import tigre
from tigre.utilities import sample_loader
from tigre.utilities import sl3d

#%% Define geometry
geo = tigre.geometry_default(high_resolution=False)


#%%
#--------------------------------------------------------------------------
#3D Shepp-Logan
#
# Type of shepp logan phantom. The shape will be the same, but Hounsfield
# values will be different
# Options:
#  phantom_type: str
#      One of {"kak-slaney", "yu-ye-wang", "toft-schabel"}, optional
#      Default is "yu-ye-wang"
#      The type of phantom.
#  size_out : int or list whose length is three, optional
#      Default is [128, 128, 128]
#      The number of voxels of phantom. [nVoxelZ, nVoxelY, nVoxelX]
#--------------------------------------------------------------------------
# phantom_type="kak-slaney"    # Air is 0. Water is 1. Proportional to Hounsfield value.
phantom_type="yu-ye-wang"    # Default of Python TIGRE Shepp-Logan phantom. Improved visual perception
# phantom_type="toft-schabel"  # Default of MATLAB TIGRE Shepp-Logan phantom.
shepp=sl3d.shepp_logan_3d(geo.nVoxel , phantom_type = phantom_type)  # Default are 128^3 and "yu-ye-wang"
tigre.plotImg(shepp, dim="z")

#%% load head phantom
head = sample_loader.load_head_phantom(geo.nVoxel)
# check the shape
print(head.shape)
# show it
plt.imshow(head, dim="z")

In [ ]:
import copy, math, sys
import numpy as np
import itk
# from itk import RTK as rtk

In [ ]:
import tigre
import numpy as np
# from tigre.utilities import sample_loader
# from tigre.utilities import CTnoise
import tigre.algorithms as algs

#%% Geometry
geo = tigre.geometry_default()

#%% Load data and generate projections
# define angles


pixel_spacing_in_micrometre = 1.9;
pixel_spacing_in_mm = pixel_spacing_in_micrometre * 1e-3;
number_of_projections = 900;
angular_span_in_degrees = 180.0;
angular_step = angular_span_in_degrees / number_of_projections;
theta_deg = np.linspace(0.,
                    angular_span_in_degrees,
                    number_of_projections,
                    endpoint=False);
theta_rad = theta_deg / 180.0 * math.pi;




print(theta_rad.shape, "angles from", theta_rad[0], "to", theta_rad[-1])

# # Load thorax phatom data
# head = sample_loader.load_head_phantom(geo.nVoxel)
# # generate projections
# projections = tigre.Ax(head, geo, angles)
# # add noise
# noise_projections = CTnoise.add(projections, Poisson=1e5, Gaussian=np.array([0, 10]))

#%% Reconstruct image using OS-SART and FDK
ReaderType = itk.ImageFileReader[CPUImageType]
reader = ReaderType.New();
reader.SetFileName("outputs/reference_normalised_projections.mha");
reader.Update();

projection_data = itk.GetArrayFromImage(reader.GetOutput())
projection_data.shape = [
    projection_data.shape[1],
    projection_data.shape[0],
    projection_data.shape[2]
]
print(projection_data.shape)



# Load thorax phatom data
head = sample_loader.load_head_phantom(geo.nVoxel)
# generate projections
projections = tigre.Ax(head, geo, theta_rad)
# add noise
projection_data = CTnoise.add(projections, Poisson=1e5, Gaussian=np.array([0, 10]))



# FDK
imgFDK = algs.fdk(projection_data, geo, theta_rad)
# OS-SART

niter = 50
imgOSSART = algs.ossart(noise_projections, geo, angles, niter)

#%% Show the results
tigre.plotimg(np.concatenate([imgFDK, imgOSSART], axis=1), dim="z")

In [ ]:
#!/usr/bin/env python
import sys
import itk
from itk import RTK as rtk


# Defines the image type
GPUImageType = rtk.Image[itk.F,3]
CPUImageType = rtk.Image[itk.F,3]

# Defines the RTK geometry object
geometry = rtk.ThreeDCircularProjectionGeometry.New()
numberOfProjections = 900
firstAngle = 0.
angularArc = 180.
sid = 140 * 1000 # source to isocenter distance (mm)
sdd = 140 * 1000 + 80 # source to detector distance (mm)

# theta_deg = np.linspace(0.,
#                     angularArc,
#                     numberOfProjections,
#                     endpoint=False);

# theta_rad = theta_deg / 180.0 * math.pi;

pixel_spacing_in_micrometre = 1.9;
pixel_spacing_in_mm = pixel_spacing_in_micrometre * 1e-3;

for x in range(0,numberOfProjections):
  angle = firstAngle + x * angularArc / numberOfProjections
  geometry.AddProjection(sid,sdd,angle)

# Writing the geometry to disk
xmlWriter = rtk.ThreeDCircularProjectionGeometryXMLFileWriter.New()
xmlWriter.SetFilename ( "geometry.xml" )
xmlWriter.SetObject ( geometry );
xmlWriter.WriteFile();







In [ ]:
# Create a stack of empty projection images
ConstantImageSourceType = rtk.ConstantImageSource[GPUImageType]
constantImageSource = ConstantImageSourceType.New()
sizeOutput = [ 1024, 1024,  numberOfProjections ]
spacing = [ pixel_spacing_in_mm, pixel_spacing_in_mm, pixel_spacing_in_mm ]
origin = [
    -(sizeOutput[0] - 1) * spacing[0] / 2.,
    -(sizeOutput[1] - 1) * spacing[1] / 2.,
    0.
]
constantImageSource.SetOrigin( origin )
constantImageSource.SetSpacing( spacing )
constantImageSource.SetSize( sizeOutput )
constantImageSource.SetConstant(0.)

RBIType = rtk.RayBoxIntersectionImageFilter[CPUImageType, CPUImageType]
rbi = RBIType.New()
rbi.SetDensity(5)
rbi.SetBoxMin([-0.10, -0.10, -0.10])
rbi.SetBoxMax([0.5, 0.5, 0.5])
rbi.SetGeometry( geometry )
rbi.SetInput(constantImageSource.GetOutput())

REIType = rtk.RayEllipsoidIntersectionImageFilter[CPUImageType, CPUImageType]
rei = REIType.New()
semiprincipalaxis = [ 0.50, 0.50, 0.50]
center = [ 0, 0, 0.10]
# Set GrayScale value, axes, center...
rei.SetDensity(2)
rei.SetAngle(0)
rei.SetCenter(center)
rei.SetAxis(semiprincipalaxis)
rei.SetGeometry( geometry )
rei.SetInput(rbi.GetOutput())


# SLPFType = rtk.SheppLoganPhantomFilter[CPUImageType, CPUImageType]
# shepp_logan = SLPFType.New()
# shepp_logan.SetGeometry( geometry )
# shepp_logan.SetInput(constantImageSource.GetOutput())
# shepp_logan.Update()

print("Origin:", constantImageSource.GetOrigin())
print("Spacing:", constantImageSource.GetSpacing())
print("Shape:", constantImageSource.GetSize())


In [ ]:
ReaderType = rtk.ImageFileReader[CPUImageType]
reader = ReaderType.New();
reader.SetFileName("outputs/reference_normalised_projections.mha");
reader.Update();

projection_data = itk.GetArrayFromImage(reader.GetOutput())
projection_data.shape = [
    projection_data.shape[1],
    projection_data.shape[0],
    projection_data.shape[2]
]
print(projection_data.shape)


temp = [];

for z in range(projection_data.shape[0]):
    proj = []
    for y in range(projection_data.shape[2]):
        proj.append(projection_data[z][0])
    temp.append(proj)

temp = np.array(temp)
print(temp.shape)

# # Graft the projections to an itk::CudaImage
# projections = GPUImageType.New()
projections = itk.GetImageFromArray(temp)
projections.SetOrigin(constantImageSource.GetOrigin())
projections.SetSpacing(constantImageSource.GetSpacing())
# projections.SetBufferedRegion(rei.GetOutput().GetBufferedRegion())
# projections.SetRequestedRegion(rei.GetOutput().GetRequestedRegion())

print("Origin:", projections.GetOrigin())
print("Spacing:", projections.GetSpacing())
print("Shape:", projections.GetRequestedRegion().GetSize())


print("Writing output image...")
WriterType = rtk.ImageFileWriter[CPUImageType]
writer = WriterType.New();
writer.SetFileName("projections.mha");
writer.SetInput(projections);
writer.Update();

In [ ]:




# Create reconstructed image
constantImageSource2 = ConstantImageSourceType.New()
sizeOutput = [ 1024, 1024, 1 ]
spacing = [ pixel_spacing_in_mm, pixel_spacing_in_mm, pixel_spacing_in_mm ]
origin = [
    -(sizeOutput[0] - 1) * spacing[0] / 2.,
    -(sizeOutput[1] - 1) * spacing[1] / 2.,
    -(sizeOutput[2] - 1) * spacing[2] / 2.,
]
print(origin)

constantImageSource2.SetOrigin( origin )
constantImageSource2.SetSpacing( spacing )
constantImageSource2.SetSize( sizeOutput )
constantImageSource2.SetConstant(0.)



# FDK reconstruction
print("Reconstructing...")
FDKGPUType = rtk.FDKConeBeamReconstructionFilter
feldkamp = FDKGPUType.New()
feldkamp.SetInput(0, constantImageSource2.GetOutput());
feldkamp.SetInput(1, projections);
feldkamp.SetGeometry(geometry);
feldkamp.GetRampFilter().SetTruncationCorrection(0.0);
feldkamp.GetRampFilter().SetHannCutFrequency(0.0);

# Field-of-view masking
FOVFilterType = rtk.FieldOfViewImageFilter[CPUImageType, CPUImageType]
fieldofview = FOVFilterType.New()
fieldofview.SetInput(0, feldkamp.GetOutput())
fieldofview.SetProjectionsStack(rei.GetOutput())
fieldofview.SetGeometry(geometry)

# Writer
print("Writing output image...")
WriterType = rtk.ImageFileWriter[CPUImageType]
writer = WriterType.New();
writer.SetFileName("fdk.mha");
writer.SetInput(fieldofview.GetOutput());
writer.Update();

print("Done!")



In [ ]:
# Simulated Geometry
GeometryType = rtk.ThreeDCircularProjectionGeometry
geometry = GeometryType.New()

for noProj in range(0, numberOfProjections):
    angle = theta_deg[noProj]
    geometry.AddProjection(sid,
                           sdd,
                           angle);
#                            args.proj_iso_x,
#                            args.proj_iso_y,
#                            args.out_angle,
#                            args.in_angle,
#                            args.source_x,
#                            args.source_y)

geometry.SetRadiusCylindricalDetector(0)

writer = rtk.ThreeDCircularProjectionGeometryXMLFileWriter.New()
writer.SetFilename("geometry.xml")
writer.SetObject(geometry)
writer.WriteFile()

In [ ]:
# Defines the image type
GPUImageType = rtk.Image[itk.F,3]
CPUImageType = rtk.Image[itk.F,3]

# Defines the RTK geometry object
geometry = rtk.ThreeDCircularProjectionGeometry.New()
for x in range(0,numberOfProjections):
    angle = theta_deg[noProj]
    geometry.AddProjection(sid,sdd,angle)

# Writing the geometry to disk
xmlWriter = rtk.ThreeDCircularProjectionGeometryXMLFileWriter.New()
xmlWriter.SetFilename ( "geometry.xml" )
xmlWriter.SetObject ( geometry );
xmlWriter.WriteFile();

# REIType = rtk.RayEllipsoidIntersectionImageFilter[CPUImageType, CPUImageType]
# rei = REIType.New()
# semiprincipalaxis = [ 50, 50, 50]
# center = [ 0, 0, 10]
# # Set GrayScale value, axes, center...
# rei.SetDensity(2)
# rei.SetAngle(0)
# rei.SetCenter(center)
# rei.SetAxis(semiprincipalaxis)
# rei.SetGeometry( geometry )
# rei.SetInput(constantImageSource.GetOutput())

# Read the projections
ReaderType = rtk.ImageFileReader[GPUImageType]
reader = ReaderType.New();
reader.SetFileName("projections.mha");
reader.Update();
projections = reader.GetOutput();


# Create reconstructed image
ConstantImageSourceType = rtk.ConstantImageSource[GPUImageType]
constantImageSource2 = ConstantImageSourceType.New()
sizeOutput = [ 1024, 1024, 1 ]
spacing = [ projections.GetSpacing()[0], projections.GetSpacing()[0], projections.GetSpacing()[0] ]
origin = [ -sizeOutput[0] * spacing[0] / 2.0 + spacing[0] / 2,
         -sizeOutput[1] * spacing[1] / 2.0 + spacing[1] / 2,
         -sizeOutput[2] * spacing[2] / 2.0 + spacing[2] / 2]
constantImageSource2.SetOrigin( origin )
constantImageSource2.SetSpacing( spacing )
constantImageSource2.SetSize( sizeOutput )
constantImageSource2.SetConstant(0.)

print("Origin:", constantImageSource2.GetOrigin())
print("Spacing:", constantImageSource2.GetSpacing())
print("Size:", constantImageSource2.GetSize())


# FDK reconstruction
print("Reconstructing...")
FDKGPUType = rtk.FDKConeBeamReconstructionFilter
feldkamp = FDKGPUType.New()
feldkamp.SetInput(0, constantImageSource2.GetOutput());
feldkamp.SetInput(1, projections);
feldkamp.SetGeometry(geometry);
feldkamp.GetRampFilter().SetTruncationCorrection(0.0);
feldkamp.GetRampFilter().SetHannCutFrequency(0.0);

# # Field-of-view masking
# FOVFilterType = rtk.FieldOfViewImageFilter[CPUImageType, CPUImageType]
# fieldofview = FOVFilterType.New()
# fieldofview.SetInput(0, feldkamp.GetOutput())
# fieldofview.SetProjectionsStack(rei.GetOutput())
# fieldofview.SetGeometry(geometry)

# Writer
print("Writing output image...")
WriterType = rtk.ImageFileWriter[CPUImageType]
writer = WriterType.New();
writer.SetFileName("recons.mha");
writer.SetInput(feldkamp.GetOutput()) #fieldofview.GetOutput());
writer.Update();

print("Done!")